In [71]:
import pandas as pd
import sys
import os
import inspect
import s5_postprocess_evaluation as pe
import configparser
import s6_display_results as dr
import s4_evaluation as evl
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")
# access parent directory from notebooks directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### For the reflection example, we are taking the following usecase: mistral medium (as tst and evaluator), non-parallel data prompt, 5 shots and no-context

In [57]:
df = dr.get_input_data()
###  'cutting' the input dataframe, evaluator is mistral medium, non-parallel data only (promptID 2), 
df = df[(df.evaluator == 'mistral-medium') & (df.isParallel == False) & (df.shots == 5) & (df.prompting == 'NoContext') & (df.model == 'mistral-medium')]
df = evl.extract_explanation(df,'explanation_accuracy')
df = df.reset_index(drop = True)
df.head(3)

,fileID,user,promptID,model,shots,runID,neutral_sentence,tst_sentence_0,user_sentence,explanation_accuracy,explanation_content_preservation,explanation_fluency,score_accuracy,score_content_preservation,score_fluency,prompting,evaluator,isParallel
0,231,U1,2,mistral-medium,5,911714,I'm all about fast food. I'm having some fries...,"Hey, I'm all about that fast food, y'all! I'm ...","I just love fast food! I’ll have some fries, s...","Both paragraphs S1 and S2 have an informal, c...",Score: 10\nExplanation: Both S1 and S2 convey ...,Score: 9\n\nExplanation: The text is highly co...,4.0,10.0,9.0,NoContext,mistral-medium,False
1,231,U1,2,mistral-medium,5,911714,Just having this vegan hotdog from the school ...,"Ah, just grabbing this vegan hotdog from the s...",Im having this vegan hotdog at school today. I...,Both paragraphs convey a similar informal and...,Score: 10\n\nExplanation: Both paragraphs disc...,Score: 9\n\nExplanation: The text is coherent ...,3.0,10.0,9.0,NoContext,mistral-medium,False
2,202,U6,2,mistral-medium,5,209327,"I went to the ""Groene Burger"" fast food restau...","So, I just popped by the 'Groene Burger' fast ...",I went to the “Groene Burger” fast food joint....,"Paragraph S1 has a more casual, conversationa...",Score: 10\n\nExplanation: Both S1 and S2 conve...,Score: 10\nExplanation: The text is highly coh...,8.0,10.0,10.0,NoContext,mistral-medium,False


In [58]:
accuracy_loop_id = 0
prompts_dict_new_accuracy = {
    # Generic prompt
    'prompt_p1': 'You are an expert in text style transfer. Here is text T1, supposedly writen in a style of person X: {} ',
                 
    # Accuracy
    'prompt_p2': 'and here is another text T2, containing are a set of unrelated sentences (separated with ;), that are actually written by person X: {} ',
    'prompt_p3': 'How different is the conversational style in T1 and T2 on a continuous scale from 1 (completely identical styles) to 10 (completely different styles)? For scoring focus on semantics and syntax. Difference in discussed topics or contexts is irrelevant for the score. Result = . Format result as "score" and "explanation".',
}

In [59]:
## GET UPDATED ACCURACIES - MISTRAL CALLS
evl_df = pd.DataFrame()
for index, row in df.iterrows():
    evl_df = pd.concat([evl_df, evl.get_accuracy_score(prompts_dict_new_accuracy, row, 'NoContext_' + format(index), accuracy_loop_id)], ignore_index=True)
evl_df = evl.extract_explanation(evl_df,'explanation_accuracy_0')
evl_df.to_csv('f8_llm_evaluation_data/Mistral/accuracy_update_summary_loop_' + str(accuracy_loop_id) + '.csv')

In [60]:
evl_df = pd.read_csv('f8_llm_evaluation_data/Mistral/accuracy_update_summary_loop_' + str(accuracy_loop_id) + '.csv')
# evl_df = evl.extract_explanation(evl_df,'explanation_accuracy_0')
df['score_accuracy_0'] = evl_df['score_accuracy_0']
df['explanation_accuracy_0'] = evl_df['explanation_accuracy_0']

In [61]:
df['score_accuracy'].astype(float).mean(),df['score_accuracy_0'].astype(float).mean()

(4.895833333333333, 6.536458333333333)

In [62]:
df.shape,df.columns

((96, 20),
 Index(['fileID', 'user', 'promptID', 'model', 'shots', 'runID',
        'neutral_sentence', 'tst_sentence_0', 'user_sentence',
        'explanation_accuracy', 'explanation_content_preservation',
        'explanation_fluency', 'score_accuracy', 'score_content_preservation',
        'score_fluency', 'prompting', 'evaluator', 'isParallel',
        'score_accuracy_0', 'explanation_accuracy_0'],
       dtype='object'))

### Refining score_accuracy_0 

In [111]:
df = df[0:5]
df

,fileID,user,promptID,model,shots,runID,neutral_sentence,tst_sentence_0,user_sentence,explanation_accuracy,explanation_content_preservation,explanation_fluency,score_accuracy,score_content_preservation,score_fluency,prompting,evaluator,isParallel,score_accuracy_0,explanation_accuracy_0
0,231,U1,2,mistral-medium,5,911714,I'm all about fast food. I'm having some fries...,"Hey, I'm all about that fast food, y'all! I'm ...","I just love fast food! I’ll have some fries, s...","Both paragraphs S1 and S2 have an informal, c...",Score: 10\nExplanation: Both S1 and S2 convey ...,Score: 9\n\nExplanation: The text is highly co...,4.0,10.0,9.0,NoContext,mistral-medium,False,8.0,The conversational styles in T1 and T2 are si...
1,231,U1,2,mistral-medium,5,911714,Just having this vegan hotdog from the school ...,"Ah, just grabbing this vegan hotdog from the s...",Im having this vegan hotdog at school today. I...,Both paragraphs convey a similar informal and...,Score: 10\n\nExplanation: Both paragraphs disc...,Score: 9\n\nExplanation: The text is coherent ...,3.0,10.0,9.0,NoContext,mistral-medium,False,6.0,The conversational styles in T1 and T2 exhibi...
2,202,U6,2,mistral-medium,5,209327,"I went to the ""Groene Burger"" fast food restau...","So, I just popped by the 'Groene Burger' fast ...",I went to the “Groene Burger” fast food joint....,"Paragraph S1 has a more casual, conversationa...",Score: 10\n\nExplanation: Both S1 and S2 conve...,Score: 10\nExplanation: The text is highly coh...,8.0,10.0,10.0,NoContext,mistral-medium,False,6.0,Although both texts are written in an informa...
3,202,U6,2,mistral-medium,5,209327,This vegan fried chicken from KFC is on the sp...,"So, you know that vegan fried chicken from KFC...",I really like the vegan fried chicken form the...,Paragraph S2 is more formal and straightforwa...,Score: 10\n\nExplanation: Both S1 and S2 conve...,Score: 9\n\nExplanation: The paragraph S1 is h...,7.0,10.0,9.0,NoContext,mistral-medium,False,6.0,\n\nAlthough both texts are written in an info...
4,201,U0,2,mistral-medium,5,379944,"I went to the ""Groene Burger"" fast food restau...","So you know what, I just went to this 'Groene ...","I went to the “groene burger”, really recommen...",Both paragraphs S1 and S2 convey the same mes...,Score: 10\n\nExplanation: S1 and S2 convey the...,Score: 10\n\nExplanation: The paragraph S1 is ...,3.0,10.0,10.0,NoContext,mistral-medium,False,7.0,Although both texts are written in an informa...


In [112]:
refine_loop_id = 0
refine_prompt_dict = {
    # System prompt
    'prompt_p1': '''You are an AI model that improves a text style transfer based on provided feedback. 
    Here is text T1, supposedly writen in a style of person X: {} ''',
                 
    # Accuracy
    'prompt_p2': '''and here is another text T2, containing are a set of unrelated sentences (separated with ;),
    that are actually written by person X: {} ''',

    # Inference
    'prompt_p3': '''Use the following feedback on text style difference between T1 and T2: {},
    to rewrite T1 so that it is more similar in style to T2. Keep the style conversational and informal.
    Result = . Format result as "sentence" and "explanation".''',
}

In [115]:
feedback_df = pd.DataFrame()
for index, row in df.iterrows():
    feedback_df = pd.concat([feedback_df, evl.get_refinement_feedback(refine_prompt_dict, row, 'NoContext_' + format(index),refine_loop_id)], ignore_index=True)
feedback_df = evl.extract_feedback(feedback_df,refine_loop_id)
feedback_df.to_csv('f8_llm_evaluation_data/Mistral/' +  "refine_summary_loop_" + str(refine_loop_id) + '.csv')
feedback_df

Refinement Feedback Prompt Query:  You are an AI model that improves a text style transfer based on provided feedback. 
    Here is text T1, supposedly writen in a style of person X: {Hey, I'm all about that fast food, y'all! I'm chowing down on some fries, a soy burger, and a coke over at the burger joint near school. And come the weekend, I'm all about the football game!} and here is another text T2, containing are a set of unrelated sentences (separated with ;),
    that are actually written by person X: {ah I think its fine! this is probably related to their privacy statement; tell me about your family, cos we are running out of time; i ll do the export, so yeah we are done!; im curious, I ll start with some analysis later. thanks Kim!!; no skiing for me, I never learned it. I think im a bit scared to do that} Use the following feedback on text style difference between T1 and T2: { The conversational styles in T1 and T2 are significantly different in terms of semantics and syntax, 

,fileID,user,promptID,model,shots,runID,neutral_sentence,tst_sentence_0,user_sentence,explanation_accuracy,...,score_accuracy,score_content_preservation,score_fluency,prompting,evaluator,isParallel,score_accuracy_0,explanation_accuracy_0,tst_sentence_1,explanation_tst_feedback_0
0,231,U1,2,mistral-medium,5,911714,I'm all about fast food. I'm having some fries...,"Hey, I'm all about that fast food, y'all! I'm ...","I just love fast food! I’ll have some fries, s...","Both paragraphs S1 and S2 have an informal, c...",...,4.0,10.0,9.0,NoContext,mistral-medium,False,8.0,The conversational styles in T1 and T2 are si...,"Hey, I reckon the privacy statement might be w...","Sentence: ""Hey, I reckon the privacy statement..."
1,231,U1,2,mistral-medium,5,911714,Just having this vegan hotdog from the school ...,"Ah, just grabbing this vegan hotdog from the s...",Im having this vegan hotdog at school today. I...,Both paragraphs convey a similar informal and...,...,3.0,10.0,9.0,NoContext,mistral-medium,False,6.0,The conversational styles in T1 and T2 exhibi...,Grabbing a vegan hotdog from the school cantee...,"Sentence: ""Grabbing a vegan hotdog from the sc..."
2,202,U6,2,mistral-medium,5,209327,"I went to the ""Groene Burger"" fast food restau...","So, I just popped by the 'Groene Burger' fast ...",I went to the “Groene Burger” fast food joint....,"Paragraph S1 has a more casual, conversationa...",...,8.0,10.0,10.0,NoContext,mistral-medium,False,6.0,Although both texts are written in an informa...,"So, I just swung by the 'Groene Burger' fast ...","Sentence: So, I just swung by the 'Groene Burg..."
3,202,U6,2,mistral-medium,5,209327,This vegan fried chicken from KFC is on the sp...,"So, you know that vegan fried chicken from KFC...",I really like the vegan fried chicken form the...,Paragraph S2 is more formal and straightforwa...,...,7.0,10.0,9.0,NoContext,mistral-medium,False,6.0,\n\nAlthough both texts are written in an info...,"So, turns out that vegan fried chicken from K...","Sentence: So, turns out that vegan fried chick..."
4,201,U0,2,mistral-medium,5,379944,"I went to the ""Groene Burger"" fast food restau...","So you know what, I just went to this 'Groene ...","I went to the “groene burger”, really recommen...",Both paragraphs S1 and S2 convey the same mes...,...,3.0,10.0,10.0,NoContext,mistral-medium,False,7.0,Although both texts are written in an informa...,"So, you know what, have you tried that ""Groen...","Sentence: So, you know what, have you tried th..."


### Evaluate the feedback - regenerate accuracy

In [121]:
accuracy_loop_id = 1
feedback_tst_evl_df = pd.DataFrame()
for index, row in feedback_df.iterrows():
    feedback_tst_evl_df = pd.concat([feedback_tst_evl_df,evl.get_accuracy_score(prompts_dict_new_accuracy, row, 'NoContext_' + format(index), accuracy_loop_id)], ignore_index=True)

Update Accuracy Prompt Query:  You are an expert in text style transfer. Here is text T1, supposedly writen in a style of person X: {Hey, I reckon the privacy statement might be why they're acting this way; I'm about to dig into some fries and a soy burger at the burger joint near school, and wash it down with a coke. Once the weekend rolls around, I'll be analyzing the football game!} and here is another text T2, containing are a set of unrelated sentences (separated with ;), that are actually written by person X: {ah I think its fine! this is probably related to their privacy statement; tell me about your family, cos we are running out of time; i ll do the export, so yeah we are done!; im curious, I ll start with some analysis later. thanks Kim!!; no skiing for me, I never learned it. I think im a bit scared to do that} How different is the conversational style in T1 and T2 on a continuous scale from 1 (completely identical styles) to 10 (completely different styles)? For scoring foc

In [126]:
feedback_tst_evl_df = evl.extract_explanation(feedback_tst_evl_df,'explanation_accuracy_' + str(accuracy_loop_id))
feedback_tst_evl_df

,fileID,user,promptID,model,shots,runID,neutral_sentence,tst_sentence_0,user_sentence,explanation_accuracy,...,score_fluency,prompting,evaluator,isParallel,score_accuracy_0,explanation_accuracy_0,tst_sentence_1,explanation_tst_feedback_0,score_accuracy_1,explanation_accuracy_1
0,231,U1,2,mistral-medium,5,911714,I'm all about fast food. I'm having some fries...,"Hey, I'm all about that fast food, y'all! I'm ...","I just love fast food! I’ll have some fries, s...","Both paragraphs S1 and S2 have an informal, c...",...,9.0,NoContext,mistral-medium,False,8.0,The conversational styles in T1 and T2 are si...,"Hey, I reckon the privacy statement might be w...","Sentence: ""Hey, I reckon the privacy statement...",3,The conversational style in T1 and T2 is quit...
1,231,U1,2,mistral-medium,5,911714,Just having this vegan hotdog from the school ...,"Ah, just grabbing this vegan hotdog from the s...",Im having this vegan hotdog at school today. I...,Both paragraphs convey a similar informal and...,...,9.0,NoContext,mistral-medium,False,6.0,The conversational styles in T1 and T2 exhibi...,Grabbing a vegan hotdog from the school cantee...,"Sentence: ""Grabbing a vegan hotdog from the sc...",7,"The two texts, T1 and T2, exhibit notable dif..."
2,202,U6,2,mistral-medium,5,209327,"I went to the ""Groene Burger"" fast food restau...","So, I just popped by the 'Groene Burger' fast ...",I went to the “Groene Burger” fast food joint....,"Paragraph S1 has a more casual, conversationa...",...,10.0,NoContext,mistral-medium,False,6.0,Although both texts are written in an informa...,"So, I just swung by the 'Groene Burger' fast ...","Sentence: So, I just swung by the 'Groene Burg...",3,The conversational styles in T1 and T2 are qu...
3,202,U6,2,mistral-medium,5,209327,This vegan fried chicken from KFC is on the sp...,"So, you know that vegan fried chicken from KFC...",I really like the vegan fried chicken form the...,Paragraph S2 is more formal and straightforwa...,...,9.0,NoContext,mistral-medium,False,6.0,\n\nAlthough both texts are written in an info...,"So, turns out that vegan fried chicken from K...","Sentence: So, turns out that vegan fried chick...",6,The conversational styles in T1 and T2 differ...
4,201,U0,2,mistral-medium,5,379944,"I went to the ""Groene Burger"" fast food restau...","So you know what, I just went to this 'Groene ...","I went to the “groene burger”, really recommen...",Both paragraphs S1 and S2 convey the same mes...,...,10.0,NoContext,mistral-medium,False,7.0,Although both texts are written in an informa...,"So, you know what, have you tried that ""Groen...","Sentence: So, you know what, have you tried th...",6,\n\nThe conversational styles in T1 and T2 dif...


In [127]:
feedback_tst_evl_df.to_csv('f8_llm_evaluation_data/Mistral/accuracy_update_summary_loop_' + str(accuracy_loop_id) + '.csv')

In [128]:
df['score_accuracy'].astype(float).mean(),df['score_accuracy_0'].astype(float).mean(),feedback_tst_evl_df['score_accuracy_1'].astype(float).mean()

(5.0, 6.6, 5.0)

# Second loop

In [130]:
refine_loop_id = 1

In [131]:
feedback_df = pd.DataFrame()
for index, row in feedback_tst_evl_df.iterrows():
    feedback_df = pd.concat([feedback_df, evl.get_refinement_feedback(refine_prompt_dict, row, 'NoContext_' + format(index),refine_loop_id)], ignore_index=True)
feedback_df = evl.extract_feedback(feedback_df,refine_loop_id)
feedback_df.to_csv('f8_llm_evaluation_data/Mistral/' +  "refine_summary_loop_" + str(refine_loop_id) + '.csv')
feedback_df

Refinement Feedback Prompt Query:  You are an AI model that improves a text style transfer based on provided feedback. 
    Here is text T1, supposedly writen in a style of person X: {Hey, I reckon the privacy statement might be why they're acting this way; I'm about to dig into some fries and a soy burger at the burger joint near school, and wash it down with a coke. Once the weekend rolls around, I'll be analyzing the football game!} and here is another text T2, containing are a set of unrelated sentences (separated with ;),
    that are actually written by person X: {ah I think its fine! this is probably related to their privacy statement; tell me about your family, cos we are running out of time; i ll do the export, so yeah we are done!; im curious, I ll start with some analysis later. thanks Kim!!; no skiing for me, I never learned it. I think im a bit scared to do that} Use the following feedback on text style difference between T1 and T2: { The conversational style in T1 and T2 

In [137]:
feedback_df

,fileID,user,promptID,model,shots,runID,neutral_sentence,tst_sentence_0,user_sentence,explanation_accuracy,...,evaluator,isParallel,score_accuracy_0,explanation_accuracy_0,tst_sentence_1,explanation_tst_feedback_0,score_accuracy_1,explanation_accuracy_1,tst_sentence_2,explanation_tst_feedback_1
0,231,U1,2,mistral-medium,5,911714,I'm all about fast food. I'm having some fries...,"Hey, I'm all about that fast food, y'all! I'm ...","I just love fast food! I’ll have some fries, s...","Both paragraphs S1 and S2 have an informal, c...",...,mistral-medium,False,8.0,The conversational styles in T1 and T2 are si...,"Hey, I reckon the privacy statement might be w...","Sentence: ""Hey, I reckon the privacy statement...",3,The conversational style in T1 and T2 is quit...,"Hey, I guess the privacy statement could be w...","Sentence: Hey, I guess the privacy statement c..."
1,231,U1,2,mistral-medium,5,911714,Just having this vegan hotdog from the school ...,"Ah, just grabbing this vegan hotdog from the s...",Im having this vegan hotdog at school today. I...,Both paragraphs convey a similar informal and...,...,mistral-medium,False,6.0,The conversational styles in T1 and T2 exhibi...,Grabbing a vegan hotdog from the school cantee...,"Sentence: ""Grabbing a vegan hotdog from the sc...",7,"The two texts, T1 and T2, exhibit notable dif...","Vegan hotdog from the canteen, not too shabby!...","Sentence: ""Vegan hotdog from the canteen, not ..."
2,202,U6,2,mistral-medium,5,209327,"I went to the ""Groene Burger"" fast food restau...","So, I just popped by the 'Groene Burger' fast ...",I went to the “Groene Burger” fast food joint....,"Paragraph S1 has a more casual, conversationa...",...,mistral-medium,False,6.0,Although both texts are written in an informa...,"So, I just swung by the 'Groene Burger' fast ...","Sentence: So, I just swung by the 'Groene Burg...",3,The conversational styles in T1 and T2 are qu...,"So, I stopped by the 'Groene Burger' fast foo...","Sentence: So, I stopped by the 'Groene Burger'..."
3,202,U6,2,mistral-medium,5,209327,This vegan fried chicken from KFC is on the sp...,"So, you know that vegan fried chicken from KFC...",I really like the vegan fried chicken form the...,Paragraph S2 is more formal and straightforwa...,...,mistral-medium,False,6.0,\n\nAlthough both texts are written in an info...,"So, turns out that vegan fried chicken from K...","Sentence: So, turns out that vegan fried chick...",6,The conversational styles in T1 and T2 differ...,"So, turns out that vegan fried chicken from K...","Sentence: So, turns out that vegan fried chick..."
4,201,U0,2,mistral-medium,5,379944,"I went to the ""Groene Burger"" fast food restau...","So you know what, I just went to this 'Groene ...","I went to the “groene burger”, really recommen...",Both paragraphs S1 and S2 convey the same mes...,...,mistral-medium,False,7.0,Although both texts are written in an informa...,"So, you know what, have you tried that ""Groen...","Sentence: So, you know what, have you tried th...",6,\n\nThe conversational styles in T1 and T2 dif...,"So, you know, there's this place called ""Groe...","Sentence: So, you know, there's this place cal..."


### evaluate this second loop

In [136]:
accuracy_loop_id = 2
feedback_tst_evl_df = pd.DataFrame()
for index, row in feedback_df.iterrows():
    feedback_tst_evl_df = pd.concat([feedback_tst_evl_df,evl.get_accuracy_score(prompts_dict_new_accuracy, row, 'NoContext_' + format(index), accuracy_loop_id)], ignore_index=True)
feedback_tst_evl_df = evl.extract_explanation(feedback_tst_evl_df,'explanation_accuracy_' + str(accuracy_loop_id))
feedback_tst_evl_df.to_csv('f8_llm_evaluation_data/Mistral/accuracy_update_summary_loop_' + str(accuracy_loop_id) + '.csv')
feedback_tst_evl_df

Update Accuracy Prompt Query:  You are an expert in text style transfer. Here is text T1, supposedly writen in a style of person X: { Hey, I guess the privacy statement could be why they're acting this way; I'm about to have some fries and a soy burger at the nearby joint, then wash it down with a coke. Once the weekend comes, I'll start analyzing the football game!} and here is another text T2, containing are a set of unrelated sentences (separated with ;), that are actually written by person X: {ah I think its fine! this is probably related to their privacy statement; tell me about your family, cos we are running out of time; i ll do the export, so yeah we are done!; im curious, I ll start with some analysis later. thanks Kim!!; no skiing for me, I never learned it. I think im a bit scared to do that} How different is the conversational style in T1 and T2 on a continuous scale from 1 (completely identical styles) to 10 (completely different styles)? For scoring focus on semantics and

,fileID,user,promptID,model,shots,runID,neutral_sentence,tst_sentence_0,user_sentence,explanation_accuracy,...,score_accuracy_0,explanation_accuracy_0,tst_sentence_1,explanation_tst_feedback_0,score_accuracy_1,explanation_accuracy_1,tst_sentence_2,explanation_tst_feedback_1,score_accuracy_2,explanation_accuracy_2
0,231,U1,2,mistral-medium,5,911714,I'm all about fast food. I'm having some fries...,"Hey, I'm all about that fast food, y'all! I'm ...","I just love fast food! I’ll have some fries, s...","Both paragraphs S1 and S2 have an informal, c...",...,8.0,The conversational styles in T1 and T2 are si...,"Hey, I reckon the privacy statement might be w...","Sentence: ""Hey, I reckon the privacy statement...",3,The conversational style in T1 and T2 is quit...,"Hey, I guess the privacy statement could be w...","Sentence: Hey, I guess the privacy statement c...",3,\nThe conversational style in T1 and T2 is qui...
1,231,U1,2,mistral-medium,5,911714,Just having this vegan hotdog from the school ...,"Ah, just grabbing this vegan hotdog from the s...",Im having this vegan hotdog at school today. I...,Both paragraphs convey a similar informal and...,...,6.0,The conversational styles in T1 and T2 exhibi...,Grabbing a vegan hotdog from the school cantee...,"Sentence: ""Grabbing a vegan hotdog from the sc...",7,"The two texts, T1 and T2, exhibit notable dif...","Vegan hotdog from the canteen, not too shabby!...","Sentence: ""Vegan hotdog from the canteen, not ...",6,The conversational style in T1 and T2 differ ...
2,202,U6,2,mistral-medium,5,209327,"I went to the ""Groene Burger"" fast food restau...","So, I just popped by the 'Groene Burger' fast ...",I went to the “Groene Burger” fast food joint....,"Paragraph S1 has a more casual, conversationa...",...,6.0,Although both texts are written in an informa...,"So, I just swung by the 'Groene Burger' fast ...","Sentence: So, I just swung by the 'Groene Burg...",3,The conversational styles in T1 and T2 are qu...,"So, I stopped by the 'Groene Burger' fast foo...","Sentence: So, I stopped by the 'Groene Burger'...",3,The conversational style in T1 and T2 is quit...
3,202,U6,2,mistral-medium,5,209327,This vegan fried chicken from KFC is on the sp...,"So, you know that vegan fried chicken from KFC...",I really like the vegan fried chicken form the...,Paragraph S2 is more formal and straightforwa...,...,6.0,\n\nAlthough both texts are written in an info...,"So, turns out that vegan fried chicken from K...","Sentence: So, turns out that vegan fried chick...",6,The conversational styles in T1 and T2 differ...,"So, turns out that vegan fried chicken from K...","Sentence: So, turns out that vegan fried chick...",3,\n\nDespite the difference in topics discussed...
4,201,U0,2,mistral-medium,5,379944,"I went to the ""Groene Burger"" fast food restau...","So you know what, I just went to this 'Groene ...","I went to the “groene burger”, really recommen...",Both paragraphs S1 and S2 convey the same mes...,...,7.0,Although both texts are written in an informa...,"So, you know what, have you tried that ""Groen...","Sentence: So, you know what, have you tried th...",6,\n\nThe conversational styles in T1 and T2 dif...,"So, you know, there's this place called ""Groe...","Sentence: So, you know, there's this place cal...",7,The conversational styles in T1 and T2 are qu...


In [138]:
df['score_accuracy'].astype(float).mean(),df['score_accuracy_0'].astype(float).mean(),feedback_tst_evl_df['score_accuracy_1'].astype(float).mean(),feedback_tst_evl_df['score_accuracy_2'].astype(float).mean()

(5.0, 6.6, 5.0, 4.4)

In [140]:
import s6_display_results as dr
col_display = ['neutral_sentence', 'user_sentence','tst_sentence_0','tst_sentence_1','tst_sentence_2']

In [141]:
dr.display_interactive_dataframe(feedback_tst_evl_df,col_display)

interactive(children=(Dropdown(description='User:', index=1, options=('All', 'U1', 'U6', 'U0'), value='U1'), D…